In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install cleverhans==3.0.1 tensorflow==1.8.0

     |████████████████████████████████| 204kB 6.3MB/s 
     |████████████████████████████████| 49.1MB 24.8MB/s 
     |████████████████████████████████| 163kB 38.1MB/s 
     |████████████████████████████████| 51kB 21.7MB/s 
     |████████████████████████████████| 3.1MB 38.5MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107221 sha256=a04823efa73f6867faa44a80e9b9220d7d7986e87bcc81f073c219583e3ace6b
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.8.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found exis

In [0]:
home_dir = '/content/gdrive/My Drive/Colab Notebooks/newlogic'
import sys
sys.path.append(home_dir)

In [0]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [0]:
from cleverhans.attacks import FastGradientMethod
from cleverhans.model import CallableModelWrapper
from cleverhans.utils import AccuracyReport
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf

In [0]:
import torch
from torchvision import datasets, transforms

In [0]:
BATCH_SIZE = 128

In [0]:
def evaluate_model(path):
    torch_model_orig = torch.load(path)
    torch_model = lambda x: torch_model_orig(x)[0] # to standard format
    with tf.Session() as sess:
        x_op = tf.placeholder(tf.float32, shape=(None, 1, 28, 28,))    
        tf_model_fn = convert_pytorch_model_to_tf(torch_model, out_dims=10)    
        cleverhans_model = CallableModelWrapper(tf_model_fn, output_layer='logits')
        # Create an FGSM attack
        fgsm_op = FastGradientMethod(cleverhans_model, sess=sess)
        fgsm_params = {'eps': 0.3, 'clip_min': 0., 'clip_max': 1.}
        adv_x_op = fgsm_op.generate(x_op, **fgsm_params)
        adv_preds_op = tf_model_fn(adv_x_op)
        test_loader = torch.utils.data.DataLoader(
          datasets.MNIST(home_dir + '/mnist', train=False, transform=transforms.ToTensor()),
          batch_size=BATCH_SIZE)
        # Run an evaluation of our model against fgsm
        total = 0
        correct = 0
        for xs, ys in tqdm(test_loader):
            adv_preds = sess.run(adv_preds_op, feed_dict={x_op: xs})
            correct += (np.argmax(adv_preds, axis=1) == ys.cpu().detach().numpy()).sum()
            total += test_loader.batch_size
    acc = float(correct) / total
    print('Adv accuracy: {:.3f}'.format(acc * 100))

In [13]:
evaluate_model(home_dir + '/output/cnn-mnist-relu.pkl')

100%|██████████| 79/79 [00:02<00:00, 30.72it/s]

Adv accuracy: 9.108


In [14]:
evaluate_model(home_dir + '/output/cnn-mnist-relu-strictening_0.1.pkl')

100%|██████████| 79/79 [00:02<00:00, 33.17it/s]

Adv accuracy: 14.290


In [15]:
evaluate_model(home_dir + '/output/cnn-mnist-relu-strictening_5.pkl')

100%|██████████| 79/79 [00:02<00:00, 33.05it/s]

Adv accuracy: 26.048


In [16]:
evaluate_model(home_dir + '/output/cnn-mnist-relog.pkl')

100%|██████████| 79/79 [00:02<00:00, 29.35it/s]

Adv accuracy: 49.318


In [17]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-more-neurons.pkl')

100%|██████████| 79/79 [00:04<00:00, 16.21it/s]

Adv accuracy: 51.839


In [18]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-maxout.pkl')

100%|██████████| 79/79 [00:04<00:00, 19.49it/s]

Adv accuracy: 72.468


In [19]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-maxout-sigmoid-out.pkl')

100%|██████████| 79/79 [00:03<00:00, 22.79it/s]

Adv accuracy: 78.550


In [21]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-minmaxout_4_2-sigmoid-out.pkl')

100%|██████████| 79/79 [00:06<00:00, 12.97it/s]

Adv accuracy: 90.872


In [22]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-minmaxout_4_2-sigmoid-out-strictening_1.pkl')

100%|██████████| 79/79 [00:06<00:00, 12.78it/s]

Adv accuracy: 58.169


In [23]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-minmaxout_4_2-sigmoid-out-strictening_0.1.pkl')

100%|██████████| 79/79 [00:06<00:00, 12.76it/s]

Adv accuracy: 82.842


In [25]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-minmaxout_4_2-sigmoid-out-strictening_0.01.pkl')

100%|██████████| 79/79 [00:06<00:00, 12.77it/s]

Adv accuracy: 88.172


In [29]:
evaluate_model(home_dir + '/output/cnn-mnist-relog-minmaxout_4_2-sigmoid-out-strictening_0.001.pkl')

100%|██████████| 79/79 [00:06<00:00, 12.72it/s]

Adv accuracy: 89.003
